In [35]:
# !pip install langchain_groq langchain_huggingface langchain_pinecone langchain_core

**llm model**

In [2]:
from langchain_groq import ChatGroq

In [3]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
llm=ChatGroq(model_name="Gemma2-9b-It")

In [5]:
llm.invoke("hi").content

'Hello! 👋 How can I help you today? 😊\n'

**HuggingFace Embeddings**

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
huggingface_embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
import pandas as pd
from langchain_core.documents import Document

In [8]:
df=pd.read_csv("/content/flipkart_product_review.csv")

In [9]:
df.head()

,product_id,product_title,rating,summary,review
0,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,1-more flexible2-bass is very high3-sound clar...
1,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Super sound and good looking I like that prize
2,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,Very much satisfied with the device at this pr...
3,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Super!,"Nice headphone, bass was very good and sound i..."
4,ACCFZGAQJGYCYDCM,BoAt Rockerz 235v2 with ASAP charging Version ...,5,Terrific purchase,Sound quality super battery backup super quali...


In [10]:
product_list = []

for _, row in df.iterrows():
    product_entry = {
        "product_name": row['product_title'],
        "product_rating": row['rating'],
        "product_summary": row['summary'],
        "product_review": row['review']
    }
    product_list.append(product_entry)

In [11]:
product_list[10]

{'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset',
 'product_rating': 5,
 'product_summary': 'Simply awesome',
 'product_review': "It's a very good product ... I'm so happy with this BCz its bass quality & sounds is too good & batry back up Also super"}

### Data Transformation

In [12]:
documents = []
for entry in product_list:
    metadata = {
        "product_name": entry["product_name"],
        "product_rating": entry["product_rating"],
        "product_summary": entry["product_summary"]
    }
    doc = Document(page_content=entry["product_review"], metadata=metadata)
    documents.append(doc)

In [13]:
documents[10]

Document(metadata={'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset', 'product_rating': 5, 'product_summary': 'Simply awesome'}, page_content="It's a very good product ... I'm so happy with this BCz its bass quality & sounds is too good & batry back up Also super")

### Pinecone VectorStore

In [14]:
import getpass
import os

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [15]:
import time

index_name = "customersupport"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [16]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=huggingface_embeddings)

In [17]:
vector_store.add_documents(documents=documents)

['2661196b-6868-4e4c-bb7f-0c21e5f54d58',
 '114c0fcc-1b3d-49c6-82f5-f52751755775',
 'a65406be-9824-436f-b054-1265942c0af5',
 'eb7ae353-7462-4875-aafc-647f96bd2cb7',
 'f9019e2e-c4d6-4f1c-bd23-9b1c6c37d392',
 '10da73de-3e98-4533-8e8d-18a764cedded',
 '87a37512-9809-4c3b-a701-eb16071c83df',
 '6aa9f995-efbb-4e37-940d-b4697f6c4f93',
 'ca6d82c4-7565-43f9-8a97-057febb3eb10',
 '27afe36a-65b0-492e-9416-b66dfe2c7a43',
 '845b819d-c669-4fb3-926a-42c0d25b8f6a',
 'b121749e-1cbd-41b0-b1d6-655da182e8da',
 '69b96717-2ef5-43a5-a096-6b6fcec68b88',
 '7b8b4bc6-41b8-4a71-8615-d964770cdabc',
 'f0ae12da-1b03-417d-b89c-64d8ff04d202',
 'f0500c37-bf16-4b25-ab14-199ac3fde58a',
 'ad50a55d-b06b-4b8f-8f12-9932746d5266',
 '6bdee4d1-4032-4b65-8e4e-78f5c71b08fe',
 'f6cb58f1-5cce-4536-9f0b-e19f9c656b71',
 'fe90e901-7aff-42b1-bbc3-08e8dd8fdf9c',
 '59038a30-0767-4c39-bac5-89bdcd298128',
 '5a06666a-8fd8-4f43-a61b-dab73f7901af',
 '3c8ecc68-c9b6-4f81-97c3-9c9865d165a9',
 'ad4ea161-7c20-4c20-a5b4-bb7bf8f93d89',
 '233c6d6b-5ea8-

In [18]:
vector_store.similarity_search("give me review of boat headphones")

[Document(id='af6840ce-9a59-415f-ad8c-90ff58841193', metadata={'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 4.0, 'product_summary': 'Value-for-money'}, page_content="Good earbuds at this price Range and Boat provides good product at Low budget and I think if you're looking for a earbuds that is good and your budget is Low than must Buy."),
 Document(id='5fb85e0e-e4cb-4d2b-8aa7-e8e671c7bfc6', metadata={'product_name': 'BoAt BassHeads 100 Wired Headset', 'product_rating': 5.0, 'product_summary': 'Just wow!'}, page_content="I bought 1more and boAt BassHeads 100 headphones together, sound clarity, volume, bass and treble of boAt is much better than 1More. Comfortable ear buds, Zara Hatke design, sleek packaging. You don't wanna listen to music without boAt headphones. Truly price worthy."),
 Document(id='e5266598-0d0d-4ef3-9720-c45b576446e3', metadata={'product_name': 'BoAt Rockerz 235v2 with ASAP charging Version 5.0 Bluetooth Headset', 'product_rating': 5.0, 'pro

In [20]:
retriever=vector_store.as_retriever()

In [ ]:
PROMPT_TEMPLATES = {
    "product_bot": """
    You are an expert EcommerceBot specialized in product recommendations and handling customer queries.
    Analyze the provided product titles, ratings, and reviews to provide accurate, helpful responses.
    Stay relevant to the context, and keep your answers concise and informative.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER:
    """
}

### LLM Chain

In [19]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
 You are an expert EcommerceBot specialized in product recommendations and handling customer queries.
    Analyze the provided product titles, ratings, and reviews to provide accurate, helpful responses.
    Stay relevant to the context, and keep your answers concise and informative.

-----------------------
Context:
{context}
-----------------------

Question: {question}

Answer:"""


)

In [21]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # your Groq LLM instance
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [25]:
query = '''give me review of boat headphones'''
result = qa_chain.invoke(query)

In [26]:
result

{'query': 'give me review of boat headphones',
 'result': 'Boat headphones generally receive positive reviews for their sound quality, particularly bass and treble, comfort, and attractive design. \n\nUsers praise the "Zara Hatke" design and sleek packaging.  \n\nHowever, some users report issues with performance, specifically with the Airdopes 131 model, citing it as "worst in performance."  \n\nThey recommend considering alternative options like the "Flipkart smartbuy bass beatz" instead. \n\n'}

In [27]:
result["result"]

'Boat headphones generally receive positive reviews for their sound quality, particularly bass and treble, comfort, and attractive design. \n\nUsers praise the "Zara Hatke" design and sleek packaging.  \n\nHowever, some users report issues with performance, specifically with the Airdopes 131 model, citing it as "worst in performance."  \n\nThey recommend considering alternative options like the "Flipkart smartbuy bass beatz" instead. \n\n'

### Chat History

In [30]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage

In [29]:
store={}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

config = {"configurable": {"session_id": "firstchat"}}

model_with_memory=RunnableWithMessageHistory(llm,get_session_history)



In [32]:
model_with_memory.invoke([HumanMessage(content="Hi! I'm Rishabh")],config=config).content

"Hello Rishabh! 😊 \n\nIt's great to meet you too.  Is there anything I can help you with? \n"

In [33]:
model_with_memory.invoke([HumanMessage(content="Hi! what is my name")],config=config).content

'Your name is Rishabh. 😊 \n\nI remember that you told me!  Is there anything else I can help you with?\n'